# In-context learning (ICL)

In context learning is a use of a generative model, where the description of a desired task is a part of the input. 

While pre-training, the model is trained on a task of "guessing" the right word in context. This is achieved by tasks like Masked Language Modeling (MLM) or Causal Language Modeling (CLM). During these tasks, the model aquires an inherent understanding of the language. 

After pre-training, traditionaly, we would then fine-tune the model through Supervised ML for a specific task for which we need:
* Training data (input and label pairs)
* Adding a specific layer ("head") to the model relevant to our desired task
The resulting model is fit for that one specific task.

When we talk about ICL, we mean models, which were finetuned using text inputs containing description of desired task(=prompts) and text outputs from several tasks.

During inference of these models we provide the prompt. Being trained on a variety of prompts for multiple tasks, the model has a better understanding of the description of the task itself and thus may show ICL capability even on never before seen tasks.



In [20]:
import sys
!{sys.executable} -m pip install git+https://github.com/fewshot-goes-multilingual/promptsource

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/fewshot-goes-multilingual/promptsource to /tmp/pip-req-build-jaj_w8va
  Running command git clone --filter=blob:none --quiet https://github.com/fewshot-goes-multilingual/promptsource /tmp/pip-req-build-jaj_w8va
  Resolved https://github.com/fewshot-goes-multilingual/promptsource to commit d2dbfda4fc6b1a839908b9daec1431428365fc13
  Preparing metadata (setup.py) ... done


In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
 
model_path = "gaussalgo/mt5-base-priming-QA_en-cs"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [22]:
prompt = """
What is meant by: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. 
I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial"
I really had to see this for myself. The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life.
In particular she wants to focus her attentions to making some sort of documentary on what the
average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. 
"""

inputs = tokenizer([prompt], return_tensors="pt", padding=True)
outputs = model.generate(**inputs.to(model.device))
outputs_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# predictions:
outputs_str

/home/nikola/L2L_MLPrague23/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['The plot is centered around a young Swedish drama student named Lena']

## Zero-shot vs few-shot



The model might understand the task from the input, but it does not know how do we expect it to respond. Therefore, if we have the model adjusted for such use, we can show it the format of the task from a few input-output examples and see if it comprehends.

This approach is called in-context few-shot learning: In addition to the description of the task, we give the model a few input-output examples (demonstrations). Given these, the model has much easier time understanding the format of the interaction that we expect from it. The demonstration are the only lead the model has to understand the task at hand. We can see, that if we pick only examples with a negative sentiment, the model is unable to pick the correct label. 

In this setting, we need to standardize the format of prediction, so that the model can rely on it.

In [23]:
input_zero_shot = """
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
input_few_shot_not_heterogenic = """
Question: What is the sentiment of the context: positive or negative? 
Context: He said, that the consert was very dull.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: She came from school sad and lonely.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
input_few_shot = """
Question: What is the sentiment of the context: positive or negative? 
Context: He said, that the consert was very dull.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: She came from school smiling and singing.
Answer:"positive"
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
inputs = tokenizer([input_zero_shot,input_few_shot_not_heterogenic,  input_few_shot], return_tensors="pt", padding=True)
outputs = model.generate(**inputs.to(model.device))
outputs_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# predictions:
outputs_str

['happy', 'positive or negative', 'positive']

## What should the prompts look like?
For training a custom in-context learner we need text pairs of a prompt and label. While in the above example we see a unified prompt, in training it is beneficial to create multiple prompts for one task, as we want to support the models capability to understand the task by its description, not by identifying a task by a template. This diversivication should yield a benefit of having the model understanding never before seen tasks better.

In [26]:
from datasets import load_dataset
from promptsource.templates import DatasetTemplates

dataset = load_dataset('super_glue', 'boolq', split="validation[:10%]")

prompts = DatasetTemplates("super_glue/boolq")
print(prompts.all_template_names)
prompt = prompts['after_reading']


Found cached dataset super_glue (/home/nikola/.cache/huggingface/datasets/super_glue/boolq/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed)


['GPT-3 Style', 'I wonder…', 'after_reading', 'based on the following passage', 'based on the previous passage', 'could you tell me…', 'exam', 'exercise', 'valid_binary', 'yes_no_question']


## Evaluation
Let's evaluate our model on a dataset created using the promptsource library and a dataset about if the answer to a question is in the context. (The model was not trained on this dataset)

In [27]:
from tqdm import tqdm

predictions = []
references = [x==1 for x in dataset["label"]]

# Get predictions
for item in tqdm(dataset):
    model_input_string = prompt.apply(item)

    inputs = tokenizer(model_input_string,padding=True, truncation=True, return_tensors="pt")
    outputs = model.generate(**inputs.to(model.device))
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    predictions.append(response_text)

# Accuracy
correct_predictions = sum([pred == str(true) for pred, true in zip(predictions, references)])
incorrect_predictions = sum([pred != str(true) for pred, true in zip(predictions, references)])

accuracy = correct_predictions / (correct_predictions+incorrect_predictions)
print("Prediction using '%s' classifier; accuracy: %s" % (model.config.model_type, accuracy))  

 41%|████      | 133/327 [01:21<03:13,  1.00it/s]

# ✋ [Hands on] Creation of an evaluation dataset 

* Download an existing dataset and transform it into a prompt input - label pair (either by creating your own prompt or by using the promtsource library).
  * Text classification (https://huggingface.co/datasets/imdb)
  * Named Entity Recognition (https://huggingface.co/datasets/polyglot_ner/viewer/en/train)
  * Question Answering (https://huggingface.co/datasets/squad_v2)
  * or other

* Adjust the evaluation script accordingly

* Create a function which will generate a few-shot (the prompt will include few demonstrations of the same task) prompt and label pairs.

* evaluate some existing ICL models on your dataset (try both zero-shot and few-shot prompts):

  * https://huggingface.co/google/flan-t5-large
  * https://huggingface.co/allenai/mtk-instruct-3b-def-pos